In [28]:
from sheets import data, headers
import pandas as pd
import plotly.express as px

In [29]:
df = pd.DataFrame(data, columns=headers)

## 1. Team that has won the most on their city (overall / per season)

### Removing redundancies

In [30]:
df["team1"] = df["team1"].replace("Rising Pune Supergiants", "Rising Pune Supergiant")
df["team2"] = df["team2"].replace("Rising Pune Supergiants", "Rising Pune Supergiant")
df["winner"] = df["winner"].replace("Rising Pune Supergiants", "Rising Pune Supergiant")
df["city"] = df["city"].replace("Bangalore", "Bengaluru")
df["city"].dropna(inplace=True)

### Matching teams with their homeground cities

In [31]:
df["winner"].unique()

array(['Rising Pune Supergiant', 'Kolkata Knight Riders',
       'Kings XI Punjab', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad', 'Mumbai Indians', 'Delhi Daredevils',
       'Gujarat Lions', 'Chennai Super Kings', 'Rajasthan Royals',
       'Deccan Chargers', 'Pune Warriors', 'Kochi Tuskers Kerala', '',
       'Delhi Capitals'], dtype=object)

In [32]:
# Creating a dictionary with -- team: homeground/secondary homeground -- pairs

team_city = {
 'Sunrisers Hyderabad': ['Hyderabad', 'Visakhapatnam'],
 'Mumbai Indians': 'Mumbai',
 'Gujarat Lions': ['Rajkot', 'Kanpur'],
 'Rising Pune Supergiant': 'Pune',
 'Royal Challengers Bangalore': 'Bengaluru',
 'Kolkata Knight Riders': 'Kolkata',
 'Delhi Daredevils': 'Delhi',
 'Kings XI Punjab': ['Chandigarh', 'Mohali', 'Indore', 'Dharamsala'],
 'Chennai Super Kings': ['Chennai', 'Ranchi'],
 'Rajasthan Royals': ['Jaipur', 'Ahmedabad'],
 'Deccan Chargers': ['Hyderabad', 'Visakhapatnam'],
 'Kochi Tuskers Kerala': 'Kochi',
 'Pune Warriors': 'Pune',
 'Delhi Capitals': 'Delhi'
}

### Finding most wins in their respective homegrounds (for all seasons)

In [33]:
df4all = df[["id", "season", "city", "winner"]]
df4all.head()
# df4all["season"].value_counts()

,id,season,city,winner
0,2,2017,Pune,Rising Pune Supergiant
1,3,2017,Rajkot,Kolkata Knight Riders
2,4,2017,Indore,Kings XI Punjab
3,5,2017,Bengaluru,Royal Challengers Bangalore
4,6,2017,Hyderabad,Sunrisers Hyderabad


In [34]:
# # list of team_city (dictionary) values

# home = list(team_city.values())
# home

In [35]:
# # since home[] contain lists within list, we require a flat list of all its values

# homeground = []
# for sublist in home:
#     if isinstance(sublist, list):
#         for minisublist in sublist:
#             homeground.append(minisublist)
#     else:
#         homeground.append(sublist)

# homeground = list(set(homeground))  # flat list of all values of home with no repeating values present in home[]

In [36]:
# # only keep rows which have homegrounds in the city column.

# df4all = df4all[df4all["city"].isin(homeground)]

In [37]:
# Assigns a "1" to all rows where a team in "winner" column is matched with the correct homeground in "city" column

df4all["T\F"] = 0
for index, row in df4all.iterrows():
    for team in team_city:
        if (row.winner == team) and (str(row.city) in team_city[team]):
            df4all.at[index, "T\F"] = 1

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
# filters all the rows with 1 in "T\F" column i.e. rows that have the correct city and homeground combination

df_final = df4all[df4all["T\F"]==1]

In [39]:
# counts the number of wins and sorts it in descending order

df_plot = df_final.groupby(["winner"])["T\F"].count().reset_index().sort_values(by=["T\F"], ascending=False)
df_plot.reset_index()

,index,winner,T\F
0,8,Mumbai Indians,53
1,7,Kolkata Knight Riders,45
2,0,Chennai Super Kings,43
3,5,Kings XI Punjab,39
4,10,Rajasthan Royals,39
5,12,Royal Challengers Bangalore,38
6,13,Sunrisers Hyderabad,32
7,3,Delhi Daredevils,28
8,4,Gujarat Lions,5
9,11,Rising Pune Supergiant,5


In [1]:
fig = px.bar(df_plot, x="winner", y="T\F", hover_data=["T\F"], labels={"winner": "Team", "T\F":"Number of wins at homeground"})
fig.show()

NameError: name 'px' is not defined

In [41]:
year = 2009
df_plot2 = df_final.groupby(["season", "winner"])["T\F"].count().reset_index().sort_values(by=["T\F"], ascending=False)
df_plot3 = df_plot2.loc[df_plot2["season"]==year]

In [42]:
fig2 = px.bar(df_plot3, x="winner", y="T\F", hover_data=["T\F"], labels={"winner": "Team", "T\F":f"Number of wins at homeground in {year}"})
fig2.show()

In [44]:
df_final["season"].value_counts()

2013    48
2011    37
2019    34
2018    31
2012    31
2017    29
2010    29
2008    29
2015    27
2016    26
2014    19
Name: season, dtype: int64

In [46]:
df4all["season"].value_counts()

2013    76
2012    74
2011    73
2019    60
2018    60
2016    60
2014    60
2010    60
2015    59
2017    58
2008    58
2009    57
Name: season, dtype: int64

In [48]:
df4all.loc[(df4all["season"]==2009) & (df4all["T\F"]==0)]

,id,season,city,winner,T\F
116,118,2009,Cape Town,Mumbai Indians,0
117,119,2009,Cape Town,Royal Challengers Bangalore,0
118,120,2009,Cape Town,Delhi Daredevils,0
119,121,2009,Cape Town,Deccan Chargers,0
120,122,2009,Port Elizabeth,Chennai Super Kings,0
121,123,2009,Durban,Kolkata Knight Riders,0
122,124,2009,Cape Town,Deccan Chargers,0
123,125,2009,Durban,Delhi Daredevils,0
124,126,2009,Cape Town,Rajasthan Royals,0
125,127,2009,Durban,Kings XI Punjab,0
